# Using NLI to classify party stances on minimum wage

Run this code in Google Colab. To import the relevant datasets, you can manually upload them using the bar on the left-hand size (under "Files"), or connect Colab with your Google Drive.

The code is tailored to the former approach, please adjust the code to import the dataset accordingly.

In [2]:
!pip install transformers pandas

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

In [5]:
# This will check if a GPU is available, and use the CPU if one is not available. Select GPU under "Change runtime type".
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")



Device: cuda


In [4]:
# Adapt to own working directory
df = pd.read_excel("/content/minimumwage_sentence_df.xlsx")


In [ ]:
df.head()

,sentence,Unnamed: 0,speech_id_link,speaker,twfy_member_id,speaker_id,hansard_id,date,legislature,chair,...,sex,birth,death,birthplace,deathplace,session,party,constituency,id,partyfacts_id
0,"I remember when, prior to 1997, before Labour ...",2301428,uk.org.publicwhip/debate/2010-05-25b.97.0,Chris Williamson,uk.org.publicwhip/member/40201,NaN,NaN,2010-05-25,55,0.0,...,male,1956-09-16 00:00:00,NaN,"52.91667,-1.46667",NaN,55.0,Labour Party,Derby North,2301429,1516.0
1,One of the achievements of the Labour Governme...,2301428,uk.org.publicwhip/debate/2010-05-25b.97.0,Chris Williamson,uk.org.publicwhip/member/40201,NaN,NaN,2010-05-25,55,0.0,...,male,1956-09-16 00:00:00,NaN,"52.91667,-1.46667",NaN,55.0,Labour Party,Derby North,2301429,1516.0
2,From the first ever national minimum wage to t...,2301525,uk.org.publicwhip/debate/2010-05-25b.39.0,Harriet Harman,uk.org.publicwhip/member/40140,NaN,NaN,2010-05-25,55,0.0,...,female,1950-07-30 00:00:00,NaN,"51.50722,-0.1275",NaN,55.0,Labour Party,Camberwell and Peckham,2301526,1516.0
3,We will be watching them on the national minim...,2301565,uk.org.publicwhip/debate/2010-05-25b.53.2,Gerald Kaufman,uk.org.publicwhip/member/40387,NaN,NaN,2010-05-25,55,0.0,...,male,1930-06-21 00:00:00,2017-02-26 00:00:00,"53.79972,-1.54917",NaN,55.0,Labour Party,"Manchester, Gorton",2301566,1516.0
4,By providing people with the skills that emplo...,2301655,uk.org.publicwhip/debate/2010-05-26b.248.0,Gordon Birtwistle,uk.org.publicwhip/member/40132,NaN,NaN,2010-05-26,55,0.0,...,male,1943-09-06 00:00:00,NaN,"53.7434,-2.3935",NaN,55.0,Liberal Democrats,Burnley,2301656,1388.0


In [6]:
model_name = "mlburnham/Political_DEBATE_large_v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

In [7]:
pipe = pipeline("zero-shot-classification", model='mlburnham/Political_DEBATE_large_v1.0', device = device, batch_size = 32) # To use the large model

Device set to use cuda


In [8]:
samples = list(df['sentence'])

template = 'The speaker in the text {} minimum wage.'

labels = ['is neutral towards', 'supports', 'opposes']

res = pipe(
    samples,
    labels,
    hypothesis_template=template,
    multi_label=False
)

# Extract the most likely label
df['stance_label'] = [label['labels'][0] for label in res]

# Rename labels
label_map = {
    'supports': 'PRO',
    'opposes': 'CON',
    'is neutral towards': 'NEUTRAL'
}

df['stance_label'] = df['stance_label'].replace(label_map)
df.head()


,sentence,Unnamed: 0,speech_id_link,speaker,twfy_member_id,speaker_id,hansard_id,date,legislature,chair,...,birth,death,birthplace,deathplace,session,party,constituency,id,partyfacts_id,stance_label
0,"I remember when, prior to 1997, before Labour ...",2301428,uk.org.publicwhip/debate/2010-05-25b.97.0,Chris Williamson,uk.org.publicwhip/member/40201,NaN,NaN,2010-05-25,55,0.0,...,1956-09-16 00:00:00,NaN,"52.91667,-1.46667",NaN,55.0,Labour Party,Derby North,2301429,1516.0,PRO
1,One of the achievements of the Labour Governme...,2301428,uk.org.publicwhip/debate/2010-05-25b.97.0,Chris Williamson,uk.org.publicwhip/member/40201,NaN,NaN,2010-05-25,55,0.0,...,1956-09-16 00:00:00,NaN,"52.91667,-1.46667",NaN,55.0,Labour Party,Derby North,2301429,1516.0,PRO
2,From the first ever national minimum wage to t...,2301525,uk.org.publicwhip/debate/2010-05-25b.39.0,Harriet Harman,uk.org.publicwhip/member/40140,NaN,NaN,2010-05-25,55,0.0,...,1950-07-30 00:00:00,NaN,"51.50722,-0.1275",NaN,55.0,Labour Party,Camberwell and Peckham,2301526,1516.0,PRO
3,We will be watching them on the national minim...,2301565,uk.org.publicwhip/debate/2010-05-25b.53.2,Gerald Kaufman,uk.org.publicwhip/member/40387,NaN,NaN,2010-05-25,55,0.0,...,1930-06-21 00:00:00,2017-02-26 00:00:00,"53.79972,-1.54917",NaN,55.0,Labour Party,"Manchester, Gorton",2301566,1516.0,NEUTRAL
4,By providing people with the skills that emplo...,2301655,uk.org.publicwhip/debate/2010-05-26b.248.0,Gordon Birtwistle,uk.org.publicwhip/member/40132,NaN,NaN,2010-05-26,55,0.0,...,1943-09-06 00:00:00,NaN,"53.7434,-2.3935",NaN,55.0,Liberal Democrats,Burnley,2301656,1388.0,PRO


In [10]:
df.to_excel("/content/minimumwage_stance_results.xlsx", index=False)